In [14]:
from mxnet import gluon

In [15]:
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Conv2D(channels=20, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    net.add(gluon.nn.Conv2D(channels=50, kernel_size=3, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(128, activation='relu'))
    net.add(gluon.nn.Dense(10))

In [16]:
import sys
sys.path.append('..')
import d2lzh.utils as utils
ctx = utils.try_gpu()
net.initialize(ctx=ctx)
print('initialize weight on', ctx)

def accuracy(output, label):
    return nd.mean(output.argmax(axis=1)==label).asscalar()

initialize weight on gpu(0)


In [18]:


from mxnet import autograd as autograd
from mxnet import nd


batch_size = 256
train_data, test_data = utils.load_data_fashion_mnist(batch_size)

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.5})

for epoch in range(5):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data.as_in_context(ctx))
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)
        
        train_loss += nd.mean(loss).asscalar()
        train_acc +=accuracy(output, label)
    
    text_acc = utils.evaluate_accuracy(test_data, net, ctx)
    print('epoch %d. Loss: %f, train acc %f, test acc %f' % (epoch, train_loss/len(train_data),train_acc/len(train_data),test_acc))

MXNetError: [11:34:52] c:\jenkins\workspace\mxnet\mxnet\src\io\../operator/elemwise_op_common.h:135: Check failed: assign(&dattr, vec.at(i)): Incompatible attr in node  at 1-th input: expected float32, got int32